In [1]:
!pip install pandas
!pip install reportlab -i http://pypi.douban.com/simple/ --trusted-host pypi.douban.com

Looking in indexes: http://pypi.douban.com/simple/


In [2]:
import pandas as pd

In [3]:
import sys,time

In [4]:
# sys.path.append('D:\conda\envs\etiology_report')
# sys.path.append('D:\conda\envs\etiology_report\Scripts')
# sys.path.append('D:\conda\envs\etiology_report\Lib')
# sys.path.append('D:\conda\envs\etiology_report\Dlls')
# sys.path.append('D:\conda\envs\etiology_report\Lib\site-packages')

In [5]:
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm,inch,mm
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
pdfmetrics.registerFont(UnicodeCIDFont('STSong-Light'))
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import BaseDocTemplate, Frame, PageTemplate, Paragraph, Spacer, Table, LongTable
from reportlab.platypus import TableStyle, tableofcontents, PageBreak, Image, NextPageTemplate
from reportlab.lib.enums import TA_JUSTIFY, TA_LEFT, TA_CENTER, TA_RIGHT
from reportlab.lib import colors
from reportlab.graphics.shapes import Drawing
from reportlab.lib.colors import HexColor
from reportlab.graphics.shapes import Line

In [6]:
class BaseReport(object):
    def __init__(self,name):
        pdfmetrics.registerFont(TTFont('SimSun', 'simsun.ttc'))  # 默认不支持中文，需要注册字体
        pdfmetrics.registerFont(TTFont('SimSunBd', 'simfang.ttf'))
        
        self.doc = BaseDocTemplate("{}.pdf".format(name), topMargin = 3.5*cm)    #声明一个文档模版类，name就是存放pdf的地址，
        self.frames = {}  # 声明一个字典，用于存放所有的frame
        self.init_frame() # 初始化frame
        self.init_page_template()  # 初始化一个列表，用于存放所有的页面模板数据
        self.Elements = []  # 声明一个列表，用于存放所有元素
        
        self.stylesheet = getSampleStyleSheet()   # 获取样式集
        self.style_list = {}  # 声明一个字典，用户存放自定义的字体样式
        self.get_style()
        
    def get_style(self):
        """
        获取样式
        """
        name_items = {
            "Heading1":{"fontSize":12,"textColor":"black","spaceAfter":1,"fontName":"SimSun"},
            "Normal":{"fontSize":6,"textColor":"#3f3f3f","spaceAfter":1,"fontName":"SimSun"},
            "BodyText":{"fontSize":12,"textColor":"#3f3f3f","spaceAfter":1,
                        "underlineGap":1,"underlineOffset":1*mm,"underlineWidth":5*mm,"fontName":"SimSun"},
            "Italic":{"fontSize":12,"textColor":"#FFFF00","spaceAfter":1,"fontName":"SimSun"},
            "Title":{"fontSize":28,"textColor":"#0b9c9e","spaceAfter":30,"fontName":"SimSun"},
            "Heading2":{"fontSize":20,"textColor":"#3f3f3f","spaceAfter":30,"alignment":TA_CENTER,"fontName":"SimSunBd"},
            "Heading3":{"fontSize":12,"textColor":"black","spaceAfter":1,"fontName":"SimSun"},
            "Code":{"fontSize":12,"textColor":"#FFFF00","spaceAfter":1,"fontName":"SimSun"},
        }
        for name,par in name_items.items():
            st = self.stylesheet[name]
            for k,v in par.items():
                setattr(st,k,par[k])
            self.style_list[name] = st
        
        
    def add_style_sheet(self,name,para):
        """
        添加样式
        """
        para["name"] = name
        style = ParagraphStyle(**para)
        setattr(style,"fontName",'SimSun')
        self.style_list[name] = style
       
    
    def footer(self,canvas, doc):
        """
        设置页脚
        :param canvas:Canvas类型  pdf画布
        :param doc:doc类型   整个pdf文件
        """
        canvas.saveState()                                                                                    #先保存当前的画布状态
        pageNumber = ("%s" %canvas.getPageNumber())                                                           #获取当前的页码
        p = Paragraph(pageNumber, self.style_list["Normal"])
        w, h = p.wrap(3*cm, 1*cm)                                                                             #申请一块1cm大小的空间，返回值是实际使用的空间
        # p.drawOn(canvas, foot_coordinate_x, foot_coordinate_y)                                                #将页码放在指示坐标处
        p.drawOn(canvas, 300, 10)
        canvas.restoreState()
        
    def header(self,canvas, doc):
        """
        设置页眉
        :param canvas:Canvas类型  pdf画布
        :param doc:doc类型     整个pdf文件
        """
        canvas.saveState()
        p = Paragraph("<img src='%s' width='%d' height='%d'/>" %("./1.jpg", 80, 45), self.style_list["BodyText"])  #使用一个Paragraph Flowable存放图片
        w, h = p.wrap(doc.width, doc.bottomMargin)
        p.drawOn(canvas, doc.leftMargin, doc.topMargin + doc.height - 0.5*cm)                                                #放置图片
        p = Paragraph("<font size=10>广州达安临床检验中心 <br/> Guangzhou Da An Clinical Laboratory Center</font>", self.style_list["BodyText"])
        w,h = p.wrap(doc.width, doc.bottomMargin)
#         p.drawOn(canvas, doc.leftMargin+doc.width-2.2*cm, doc.topMargin+ doc.height-0.3*cm) #放置报告这句话
        p.drawOn(canvas, doc.leftMargin + 3.2*cm, doc.topMargin+ doc.height-0.3*cm)
        canvas.line(doc.leftMargin, doc.bottomMargin+doc.height + 0.5*cm, doc.leftMargin+doc.width, doc.bottomMargin+doc.height + 0.5*cm) #画一条横线
        canvas.restoreState()
        
    def init_frame(self):
        """
        初始化frame
        return:frame
        """
        frame = {}
        # 生成一个首页的frame
        frame_home = Frame(self.doc.leftMargin, self.doc.bottomMargin, self.doc.width, self.doc.height, id='frame_home')
        frame["frame_home"] = {"frame":frame_home,"onPage":None,"onPageEnd":None}
        # 生成一个正文的frame
        frame_body = Frame(self.doc.leftMargin, self.doc.bottomMargin, self.doc.width, self.doc.height, id='normal')   #声明一块Frame，存放页码
        frame["normal"] = {"frame":frame_body,"onPage":self.header,"onPageEnd":self.footer}
        self.frames.update(frame)
    
    def add_frame(self,frame_name,frame,header,footer):
        """添加一个frame"""
        frame = {}
        frame = Frame(self.doc.leftMargin, self.doc.bottomMargin, self.doc.width, self.doc.height, id=frame_name)   
        frame[frame_name] = {"frame":frame,"onPage":header,"onPageEnd":footer}
        self.frames.update(frame)
        # 初始化页面模板
        self.init_page_template()
        
    def init_page_template(self):
        """
        初始化页面模板
        """
        templates = []
        for name,para in self.frames.items():
            onPage = para.get("onPage")
            onPageEnd = para.get("onPageEnd")
            frame = para.get("frame")
            if not frame:
                continue
            if onPage and onPageEnd:
                templates.append(PageTemplate(id=name,frames=frame,onPage=onPage, onPageEnd=onPageEnd))
            else:
                templates.append(PageTemplate(id=name,frames=frame))
        self.page_templates = templates
        self.doc.addPageTemplates(self.page_templates)
    
    def add_empty_line(self):
        """添加一行空行"""
        self.Elements.append(Paragraph("", self.style_list["BodyText"]))
    
    def next_page(self):
        """
        翻到下一页
        """
        self.Elements.append(PageBreak())
        
    def create_doc(self):
        """
        创建报告文件
        """
        self.doc.build(self.Elements)
        
    def run(self):
        """
        生成报告文件
        """
        pass

In [31]:
class EtiologyReport(BaseReport):
    """病原报告"""
    def __init__(self,name):
        super().__init__(name)
        
        # 添加一个首页的基础信息样式
        base_style = {
            "fontSize":12,"textColor":"#3f3f3f","spaceAfter":20,"underlineGap":1,"underlineOffset":-1*mm,"underlineWidth":0.1*mm,
            "alignment":TA_CENTER}
        self.add_style_sheet("BaseInfo",base_style)
        
        
    def add_base_info_table(self):
        """添加一个基础信息表格"""
        table_data = [
                    ['编号：', 'XXXXXXXXXXXXX', '姓名：','XXX'],
                    ['性别：', '男', '年龄：','XX'],
                    ['样本来源：', 'XX省XX市XX区XX街道XX社区居委会\nXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX', '样本类型：','粪便'],
                    ['检测方法：', '下一代高通量测序技术\n（Illuminna）', '检测单位：','XX医院检验医学部'],
                    ['检测日期：2021.11', '', '', ''],
              ]
        table_style = [
            ('FONTNAME', (0, 0), (-1, -1), 'SimSun'),  # 字体
            ('FONTSIZE', (0, 1), (-1, -1), 10),  # 字体大小
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
            ("BOTTOMPADDING",(0, 0), (1, 1), 5),
            ('SPAN', (0, 4), (3, 4)),  # 合并
            ('BACKGROUND', (0, 0), (0, 3), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (1, 0), (1, 3), '#f0f0f0'),  # 设置背景颜色
            ('BACKGROUND', (2, 0), (2, 3), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (3, 0), (3, 3), '#f0f0f0'),  # 设置背景颜色
            ('BACKGROUND', (0, 4), (3, 4), '#e1e1e1'),  # 设置背景颜色
#             ('TEXTCOLOR', (0, -1), (0, -1), colors.green),  # 设置表格内文字颜色
            ('GRID', (0, 0), (-1, -1), 1.5, colors.white),  # 设置表格框线为灰色，线宽为0.1
            ('ALIGN', (0, 4), (3, 4) , 'RIGHT'),
            ('BOTTOMMARGING',(0, 0), (-1, -1),10)
        ]
        table = Table(data=table_data, style=table_style)
        return table
    
    def home_page(self):
        """首页"""
        self.Elements.append(NextPageTemplate("frame_home"))
        image_left = Image("./首页/top.jpg",hAlign="CENTER")
        image_left.drawWidth,image_left.drawHeight =  7.6*inch,3.25*inch
        self.Elements.append(image_left)

        title = '<font size=20 >病原微生物宏基因组（RNA）</font>'
        self.Elements.append(Paragraph(title, self.style_list["Title"]))
        title2 =  '— 检测报告 —'
        self.Elements.append(Paragraph(title2, self.style_list["Heading2"]))
        for item in range(5):
            self.Elements.append(Paragraph("", self.style_list["Title"]))
            
        #表格
        
        table_data = [
            ["受检者：", ""],
            ["送检单位：",""],
            ["检测单位：","广州达安临床检验中心"],
            ["接收日期：",""],
            ["报告日期：",""],
            
        ]
            
        table_style = [
            ('FONTNAME', (0, 0), (-1, -1), 'SimSun'),  # 字体
            ('FONTSIZE', (0, 0), (-1, -1), 12),  # 字体大小
            #('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'BOTTOM'),  # 所有表格上下居中对齐
            ('LINEBELOW',(1,0),(1,-1), 1,'#3f3f3f' ),  # 设置下边框
            
        ]
        table = Table(data=table_data, style=table_style,colWidths=(2.5*cm,8*cm),rowHeights=1.2*cm)
        self.Elements.append(table)
        
        
        
#         # 空格
#         space = []
#         for item in range(30):
#             space.append("&nbsp;")
        
#         base_info1 = "受检者：<u>{}{}{}</u>".format("".join(space),"","".join(space))
#         self.Elements.append(Paragraph(base_info1, self.style_list["BaseInfo"]))
#         base_info2 = "送检单位：<u>{}{}{}</u>".format("".join(space),"","".join(space))
#         self.Elements.append(Paragraph(base_info2, self.style_list["BaseInfo"]))
#         base_info = '广州达安临床检验中心'
#         lengh = len(space) - int(len(base_info) * 1.5)
#         base_info3 = "检测单位：<u>{}{}{}</u>".format("".join(space[:lengh]),base_info,"".join(space[:lengh]))
#         self.Elements.append(Paragraph(base_info3, self.style_list["BaseInfo"]))
#         base_info4 = "检测单位：<u>{}{}{}</u>".format("".join(space),"","".join(space))
#         self.Elements.append(Paragraph(base_info4, self.style_list["BaseInfo"]))
#         base_info5 = "报告日期：<u>{}{}{}</u>".format("".join(space),"","".join(space))
#         self.Elements.append(Paragraph(base_info5, self.style_list["BaseInfo"]))
    
    def declaration_page(self):
        """声明页"""
         # 添加一个声明页的标题样式
        declar_style = {
            "fontSize":20,"textColor":"#0b9c9e","spaceAfter":20,"underlineGap":1,"underlineOffset":-1*mm,"underlineWidth":0.1*mm,
            "alignment":TA_CENTER,"bulletFontName":"Symbol"}
        self.add_style_sheet("declarTitle",declar_style)
        title = "致客户"
        self.Elements.append(Paragraph(title, self.style_list["declarTitle"]))
        content1 = '<strong >尊敬的客户：</strong>'
        self.Elements.append(Paragraph(content1, self.style_list["BodyText"]))
        self.add_empty_line()
        content_declar = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":5,"spaceBefore":5,"firstLineIndent":24,"leading":24}
        self.add_style_sheet("content_declar",content_declar)
        contents = [
            "您好！",
            "感谢您对我们的信任。",
            "病原微生物宏基因组检测是一种不依赖传统微生物培养而广泛分析临床样本中微生物（细菌、真菌、 病毒、寄生虫等）的高通量测序检测技术。通过临床样本的中微生物总体基因组（DNA或RNA）的提取、高通量测序和数据分析，可获得疑似致病微生物的种属信息。 ",
            "达安临床检验中心本着严谨科学的态度，结合目前最新的指南、共识和临床研究现状，采用高通量测序技术和智能化算法分析，对您的样本进行病原微生物宏基因组检测，可辅助医生进行临床病原微生物诊断，致力于临床精准诊断和精准治疗。",
            "本检测报告是临床医生在临床诊断和制定治疗方案时强有力的参考依据。除此之外，还需结合患者的其他情况进行综合考虑，最终的诊断结果和治疗方案需要由主治医生进行综合判断和选择。",
            "希望您能将您的宝贵意见或建议及时反馈给我们。我们的免费服务电话是：800-830-3620（周一至周五，8:30--17:30），我们将竭诚为您服务！"
        ]
        for content in contents:
            self.Elements.append(Paragraph(content, self.style_list["content_declar"]))
        for item in range(20):
            self.add_empty_line()
        right_declar = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":5,"spaceBefore":5,"firstLineIndent":24,"leading":24, "alignment":TA_RIGHT}
        self.add_style_sheet("right_declar",right_declar)
        right_content = "广州达安临床检验中心"
        self.Elements.append(Paragraph(right_content, self.style_list["right_declar"]))
    
    def base_info(self):
        """受检者信息"""
        baseinfo_title_style = {"fontSize":14,"textColor":"#0b9c9e","spaceAfter":20,"spaceBefore":20,"alignment":TA_LEFT,"bulletFontName":"Symbol"}
        self.add_style_sheet("baseinfo_title",baseinfo_title_style)
        baseinfo_title1 = "一、受检者信息"
        self.Elements.append(Paragraph(baseinfo_title1, self.style_list["baseinfo_title"]))
        table_data = [
                    ['姓名', '', '性别','男',"年龄","69岁"],
                    ['送检医院', '', '','','',''],
                    ['送检科室', '呼吸与危重症医学 （重症监护科）', '送检医生',"-",'门诊号/住院号',"-"],
                    ['样本类型', '肺泡灌洗液', '','',"条码号",""],
                    ['临床诊断', '/', '', '',"",""],
                    ['个人病史', '/', '', '',"",""],
                    ['其他检测结果', '/', '', '',"",""],
                    ['采样日期','','接受日期','','报告日期','']
              ]
        table_style = [
           
            ('FONTNAME', (0, 0), (-1, -1), 'SimSun'),  # 字体
            ('FONTSIZE', (0, 0), (-1, -1), 10),  # 字体大小
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
#             ("BOTTOMPADDING",(0, 0), (1, 1), 5),  # 设置外边框
            ('SPAN', (1, 1), (-1, 1)),  # 合并
            ('SPAN', (1, 3), (3,  3)),  # 合并
            ('SPAN', (1, 4), (-1, 4)),  # 合并
            ('SPAN', (1, 5), (-1, 5)),  # 合并
            ('SPAN', (1, 6), (-1, 6)),  # 合并
            ('BACKGROUND', (0, 0), (0, -1), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (2, 0), (2, 0), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (2, 2), (2, 2), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (4, 0), (4, 0), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (4, 2), (4, 2), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (4, 3), (4, 3), '#e1e1e1'),  # 设置背景颜色
            ('BACKGROUND', (4, -1), (4, -1), '#e1e1e1'),  # 设置背景颜色
            ('GRID', (0, 0), (-1, -1), 0.05, colors.gray),  # 设置表格框线为灰色，线宽为0.1
            ('BOX',(0,0),(-1,-1),0.1,colors.gray),
            ('LINEABOVE',(0,0),(-1,0), 2 , "#0b9c9e"),  # 设置边框
            ('LINEBELOW',(0,-1),(-1,-1), 1 , "#0b9c9e"),  # 设置边框
        ]
        table = Table(data=table_data, style=table_style)
        #table = Table(data=table_data, style=table_style,colWidths=(2.8*cm,3*cm,2*cm,3*cm,2.5*cm,3.5*cm),rowHeights=1.2*cm)
        self.Elements.append(table)
        
        baseinfo_title2 = '二、检测结果摘要'
        self.Elements.append(Paragraph(baseinfo_title2, self.style_list["baseinfo_title"]))
        
        result_abstract_header_style = {"fontSize":14,"textColor":"#ffffff","spaceAfter":24,"spaceBefore":0,"alignment":TA_CENTER}
        self.add_style_sheet("result_abstract_header",result_abstract_header_style)
        
        result_abstract_content_style = {"fontSize":10,"textColor":"#3f3f3f","spaceAfter":0,"spaceBefore":0,"alignment":TA_CENTER,"LEADING":10}
        self.add_style_sheet("result_abstract_content",result_abstract_content_style)
        
        germ_info = [
                    "铜绿假单胞菌（95549）"," 金黄色葡萄球菌 （1661）"," 蜡样芽孢杆菌复合群 （1189）",
                     " 大肠埃希菌 （158）"," 肺炎克雷伯菌 （41）"," 常见微生态菌群 其他病原请见详情列表"
                    ]
        germ_list = []
        for germ in germ_info:
            germ_list.append(Paragraph(germ, self.style_list["result_abstract_content"]))
        
        virus_list = []
        virus_info = [
                    "人类疱疹病毒 1 型(HSV1) （9）","人类疱疹病毒 5 型(CMV)（6）"
                    ]
        for virus in virus_info:
            virus_list.append(Paragraph(virus, self.style_list["result_abstract_content"]))
        
        fungus_info = ["未检出疑似病原体","常见微生态菌群","其他病原请见详情列表"]
        fungus_list = []
        for fungus in fungus_info:
            fungus_list.append(Paragraph(fungus, self.style_list["result_abstract_content"]))
            
        parasite_info = ["未检出"]
        parasite_list = []
        for parasite in parasite_info:
            parasite_list.append(Paragraph(parasite, self.style_list["result_abstract_content"]))
            
        table_data = [
                    [Paragraph('各类疑似致病的病原微生物检出结果', self.style_list["result_abstract_header"]), '', '',''],
                    ['细菌', '病毒', '真菌','寄生虫'],
                    [germ_list, virus_list, fungus_list,parasite_list]
                   ]
        
        table_style = [
           
            ('FONTNAME', (0, 0), (-1, -1), 'SimSun'),  # 字体
            ('FONTSIZE', (0, 0), (-1, -1), 10),  # 字体大小
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
#             ("BOTTOMPADDING",(0, 0), (1, 1), 5),  # 设置外边框
            ('SPAN', (0, 0), (-1, 0)),  # 合并
            
            ('BACKGROUND', (0, 0), (-1, 0), '#0b9c9e'),  # 设置背景颜色
            ('GRID', (0, 0), (-1, -1), 0.05, colors.gray),  # 设置表格框线为灰色，线宽为0.1
            ('BOX',(0,0),(-1,-1),0.1,colors.gray),
            ('LINEABOVE',(0,0),(-1,0), 1 , "#0b9c9e"),  # 设置下边框
            ('LINEBELOW',(0,0),(-1,0), 1 , "#0b9c9e"),  # 设置下边框
            ('LINEBELOW',(0,-1),(-1,-1), 1 , "#0b9c9e"),  # 设置下边框
        ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table)
        
        notice_content = [
            "*注：",
            "1. 括号内为检出的特异性序列（reads）数；详细检测结果见 3.1-3.6。 ",
            "2. 对于免疫低下/缺陷患者，请注意关注微生态菌群（3.6）；部分重要的条件致病微生物（▲）见微生态菌群列表。",
            "3. 建议临床结合患者症状和其它辅助检测，进一步确认患者的感染情况。"
        ]
        notice_content_style = {"fontSize":10,"textColor":"#3f3f3f","spaceAfter":10,"spaceBefore":5,"alignment":TA_LEFT}
        self.add_style_sheet("notice_content",notice_content_style)
        for notice in notice_content:
            self.Elements.append(Paragraph(notice, self.style_list["notice_content"]))
            
        # 添加一条分割线
        d = Drawing(100, 1)
        line = Line(0, 0, self.doc.width, 0)
        line.strokeColor = HexColor('#0b9c9e')
        d.add(line)
        self.Elements.append(d)
        # 
        checker_content1_style = {"fontSize":12,"textColor":"#0000","spaceAfter":10,"spaceBefore":0,"alignment":TA_RIGHT}
        checker_content2_style = {"fontSize":12,"textColor":"#0000","spaceAfter":10,"spaceBefore":0,"alignment":TA_LEFT}
        self.add_style_sheet("checker_content1",checker_content1_style)
        self.add_style_sheet("checker_content2",checker_content2_style)
        table_data = [
                    [Paragraph('检测者:', self.style_list["checker_content1"]), 
                     Paragraph('黎志浩', self.style_list["checker_content2"]),
                     Paragraph('审核者:', self.style_list["checker_content1"]),
                     Paragraph('', self.style_list["checker_content2"])],
                   ]
        
        table_style = [
        
        ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table)
        self.add_empty_line()
        last_content =  '本报告仅供临床科研参考，本次检测仅对本样本负责，如有疑义请在七个工作日内与我们联系。'
        self.Elements.append(Paragraph(last_content, self.style_list["notice_content"]))
        
    
    def test_result_detail(self):
        """检测结果详述"""
        baseinfo_title3 = '三、检测结果详述'
        self.Elements.append(Paragraph(baseinfo_title3, self.style_list["baseinfo_title"]))
        
        header2_content = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":30,"alignment":TA_LEFT,"fontName":"SimSunBd"}
        self.add_style_sheet("header2_content",header2_content)
        baseinfo_title31 = '3.1 细菌检测结果'
        self.Elements.append(Paragraph(baseinfo_title31, self.style_list["header2_content"]))
        
        title2_content = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":30,"alignment":TA_CENTER,"fontName":"SimSunBd"}
        self.add_style_sheet("title2_content",title2_content)
        
        title2_info = [
            "类型","属名" ,"相对丰度%" ,"reads 数" ,"种名" ,"鉴定置信度", "reads 数"
        ]
        table_contents = [
            "G-","假单胞菌属<br></br> Pseudomonas","88.70","166040","铜绿假单胞菌<br></br>Pseudomonas aeruginosa","0.99","95549"
        ]
        
        table_data = [
                    [Paragraph('属', self.style_list["result_abstract_header"]), '', '', Paragraph('种', self.style_list["result_abstract_header"]),"",'',''],
                    [Paragraph(item, self.style_list["title2_content"]) for item in title2_info],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents]
            
                   ]
        
        table_style = [
           
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
#             ("BOTTOMPADDING",(0, 0), (1, 1), 5),  # 设置外边框
            ('SPAN', (0, 0), (2, 0)),  # 合并
            ('SPAN', (3, 0), (-1, 0)),  # 合并
            
            ('BACKGROUND', (0, 0), (-1, 0), '#0b9c9e'),  # 设置背景颜色
            ('BACKGROUND', (0, 1), (-1, 1), '#daeef3'),  # 设置背景颜色
            ('GRID', (0, 0), (-1, -1), 0.05, colors.gray),  # 设置表格框线为灰色，线宽为0.1
            ('BOX',(0,0),(-1,-1),0.1,colors.gray),
        ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table)
        
        notice_content = [
            "注：",
           " 1. G+为革兰氏染色阳性菌，G-为革兰氏染色阴性菌（下同）。",
            "2. 相对丰度：该病原体相应分类中基因组相对比例（下同）。" ,
            "3. reads 数：高通量测序序列唯一比对到某属或种微生物特异性序列数（下同）。",
            "4. 鉴定置信度：根据序列比对，在样本中鉴定该病原体的可信度（下同）。"
        ]
        
        for notice in notice_content:
            self.Elements.append(Paragraph(notice, self.style_list["notice_content"]))
            
        # 3.2
        baseinfo_title31 = '3.2 真菌检测结果'
        self.Elements.append(Paragraph(baseinfo_title31, self.style_list["header2_content"]))
        
        title2_content = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":30,"alignment":TA_CENTER,"fontName":"SimSunBd"}
        self.add_style_sheet("title2_content",title2_content)
        
        title2_info = [
            "类型","属名" ,"相对丰度%" ,"reads 数" ,"种名" ,"鉴定置信度", "reads 数"
        ]
        table_contents = [
            "-","-","-","-","-","-","-"
        ]
        
        table_data = [
                    [Paragraph('属', self.style_list["result_abstract_header"]), '', '', Paragraph('种', self.style_list["result_abstract_header"]),"",'',''],
                    [Paragraph(item, self.style_list["title2_content"]) for item in title2_info],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                   ]
        
        table_style = [
           
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
#             ("BOTTOMPADDING",(0, 0), (1, 1), 5),  # 设置外边框
            ('SPAN', (0, 0), (2, 0)),  # 合并
            ('SPAN', (3, 0), (-1, 0)),  # 合并
            
            ('BACKGROUND', (0, 0), (-1, 0), '#0b9c9e'),  # 设置背景颜色
            ('BACKGROUND', (0, 1), (-1, 1), '#daeef3'),  # 设置背景颜色
            ('GRID', (0, 0), (-1, -1), 0.05, colors.gray),  # 设置表格框线为灰色，线宽为0.1
            ('BOX',(0,0),(-1,-1),0.1,colors.gray),
        ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table)
        
       # 3.3 病毒检测结果
        baseinfo_title31 = '3.3 病毒检测结果'
        self.Elements.append(Paragraph(baseinfo_title31, self.style_list["header2_content"]))
        
        title2_content = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":30,"alignment":TA_CENTER,"fontName":"SimSunBd"}
        self.add_style_sheet("title2_content",title2_content)
        
        table_contents = [
            "人类疱疹病毒 1 型(HSV1)<br></br> Human alphaherpesvirus1"  ,"0.99"," 9"
        ]
        
        table_data = [
                    [Paragraph('名称', self.style_list["result_abstract_header"]), 
                     Paragraph('鉴定置信度（%）', self.style_list["result_abstract_header"]), 
                     Paragraph('reads 数', self.style_list["result_abstract_header"])],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                   ]
        
        table_style = [
           
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
#             ("BOTTOMPADDING",(0, 0), (1, 1), 5),  # 设置外边框
#             ('SPAN', (0, 0), (2, 0)),  # 合并
#             ('SPAN', (3, 0), (-1, 0)),  # 合并
            
            ('BACKGROUND', (0, 0), (-1, 0), '#0b9c9e'),  # 设置背景颜色
#             ('BACKGROUND', (0, 1), (-1, 1), '#daeef3'),  # 设置背景颜色
            ('GRID', (0, 0), (-1, -1), 0.05, colors.gray),  # 设置表格框线为灰色，线宽为0.1
            ('BOX',(0,0),(-1,-1),0.1,colors.gray),
        ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table) 
        
        
       # 3.4 寄生虫检测结果
        baseinfo_title31 = '3.4 寄生虫检测结果'
        self.Elements.append(Paragraph(baseinfo_title31, self.style_list["header2_content"]))
        
        title2_content = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":30,"alignment":TA_CENTER,"fontName":"SimSunBd"}
        self.add_style_sheet("title2_content",title2_content)
        
        title2_info = [
            "类型","属名" ,"相对丰度%" ,"reads 数" ,"种名" ,"鉴定置信度", "reads 数"
        ]
        table_contents = [
            "-","-","-","-","-","-","-"
        ]
        
        table_data = [
                    [Paragraph('属', self.style_list["result_abstract_header"]), '', '', Paragraph('种', self.style_list["result_abstract_header"]),"",'',''],
                    [Paragraph(item, self.style_list["title2_content"]) for item in title2_info],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                   ]
        
        table_style = [
           
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
#             ("BOTTOMPADDING",(0, 0), (1, 1), 5),  # 设置外边框
            ('SPAN', (0, 0), (2, 0)),  # 合并
            ('SPAN', (3, 0), (-1, 0)),  # 合并
            
            ('BACKGROUND', (0, 0), (-1, 0), '#0b9c9e'),  # 设置背景颜色
            ('BACKGROUND', (0, 1), (-1, 1), '#daeef3'),  # 设置背景颜色
            ('GRID', (0, 0), (-1, -1), 0.05, colors.gray),  # 设置表格框线为灰色，线宽为0.1
            ('BOX',(0,0),(-1,-1),0.1,colors.gray),
        ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table)
        
        
        #3.5 疑似人体微生态菌群检测结果
        baseinfo_title31 = '3.5 疑似人体微生态菌群检测结果'
        self.Elements.append(Paragraph(baseinfo_title31, self.style_list["header2_content"]))
        
        title2_content = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":30,"alignment":TA_CENTER,"fontName":"SimSunBd"}
        self.add_style_sheet("title2_content",title2_content)
        
        title2_info = [
            "类型","属名" ,"相对丰度%" ,"reads 数" ,"种名" ,"鉴定置信度", "reads 数"
        ]
        table_contents = [
            "G-","假单胞菌属<br></br> Pseudomonas","88.70","166040","铜绿假单胞菌<br></br>Pseudomonas aeruginosa","0.99","95549"
        ]
        
        table_data = [
                    [Paragraph('属', self.style_list["result_abstract_header"]), '', '', Paragraph('种', self.style_list["result_abstract_header"]),"",'',''],
                    [Paragraph(item, self.style_list["title2_content"]) for item in title2_info],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents]
            
                   ]
        
        table_style = [
           
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
#             ("BOTTOMPADDING",(0, 0), (1, 1), 5),  # 设置外边框
            ('SPAN', (0, 0), (2, 0)),  # 合并
            ('SPAN', (3, 0), (-1, 0)),  # 合并
            
            ('BACKGROUND', (0, 0), (-1, 0), '#0b9c9e'),  # 设置背景颜色
            ('BACKGROUND', (0, 1), (-1, 1), '#daeef3'),  # 设置背景颜色
            ('GRID', (0, 0), (-1, -1), 0.05, colors.gray),  # 设置表格框线为灰色，线宽为0.1
            ('BOX',(0,0),(-1,-1),0.1,colors.gray),
        ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table)
        
        notice_content = [
            "注：人体微生态菌群为存在于人体皮肤、呼吸道、口腔、胃肠道、泌尿道的微生物，多为条件致病菌，正常条件下与人体共生，在免疫力低下/缺陷的患者中具有潜在致病性。"
        ]
        
        for notice in notice_content:
            self.Elements.append(Paragraph(notice, self.style_list["notice_content"]))
    
    
        #3.6 疑似耐药基因检测结果
        
        baseinfo_title31 = '3.6 疑似耐药基因检测结果'
        self.Elements.append(Paragraph(baseinfo_title31, self.style_list["header2_content"]))
        
        title2_content = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":30,"alignment":TA_CENTER,"fontName":"SimSunBd"}
        self.add_style_sheet("title2_content",title2_content)
        
        table_contents = [
           "-","-","-","-","-"
        ]
        
        table_data = [
                    [Paragraph('基因名称', self.style_list["result_abstract_header"]), 
                     Paragraph('疑似对应物种', self.style_list["result_abstract_header"]), 
                     Paragraph('reads 数', self.style_list["result_abstract_header"]),
                     Paragraph('耐药类型', self.style_list["result_abstract_header"]), 
                     Paragraph('相关物种', self.style_list["result_abstract_header"])],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                   ]
        
        table_style = [
           
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
#             ("BOTTOMPADDING",(0, 0), (1, 1), 5),  # 设置外边框
#             ('SPAN', (0, 0), (2, 0)),  # 合并
#             ('SPAN', (3, 0), (-1, 0)),  # 合并
            
            ('BACKGROUND', (0, 0), (-1, 0), '#0b9c9e'),  # 设置背景颜色
#             ('BACKGROUND', (0, 1), (-1, 1), '#daeef3'),  # 设置背景颜色
            ('GRID', (0, 0), (-1, -1), 0.05, colors.gray),  # 设置表格框线为灰色，线宽为0.1
            ('BOX',(0,0),(-1,-1),0.1,colors.gray),
        ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table) 
        
        notice_content = [
            "注：耐药基因与表型之间可能存在差异，检出耐药基因并不能明确该菌一定对相应药物耐药，检测结果仅供参考，请以临床医生指导用药为准。更多耐药基因信息详见：https://cge.cbs.dtu.dk/services/ResFinder/；https://card.mcmaster.ca/"
        ]
        
        for notice in notice_content:
            self.Elements.append(Paragraph(notice, self.style_list["notice_content"]))
        
    
    def test_by_declare(self):
        """检出病原体说明"""
        baseinfo_title4 = '四、检出病原体说明'
        self.Elements.append(Paragraph(baseinfo_title4, self.style_list["baseinfo_title"]))

        content_declar = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":5,"spaceBefore":5,"firstLineIndent":0,"leading":24}
        self.add_style_sheet("content_declar",content_declar)
        contents = [
            "铜绿假单胞菌 (Pseudomonas aeruginosa)：革兰阴性杆菌，假单胞菌属。铜绿假单胞菌广泛分布于水、空气、土壤，以及正常人体皮肤、呼吸道与肠道粘膜中，为条件致病菌。因手术、化疗、放疗、激素治疗等原因人体抵抗力低下时容易引起感染。可引起烧伤创面感染、肺部感染、泌尿道感染、中耳炎、脑膜炎、菌血症等。是院内感染的常见病原体。",
            
        ]
        for content in contents:
            self.Elements.append(Paragraph(content, self.style_list["content_declar"]))
        
        #插入图片
        image41 = Image('4-1.jpg',hAlign='CENTER',width = 10 * cm,height = 6.8 * cm)  #设置图片尺寸，居中
        self.Elements.append(image41)
        
        
        
        notice_content = [
            "注：以上所展示的覆盖度图为针对特定微生物绘制图谱，反映比对到该微生物的序列在其基因组上的分布情况，横坐标代表该微生物的基因组大小，纵坐标代表不同基因组区段内检出的序列数。"
        ]
        for notice in notice_content:
            self.Elements.append(Paragraph(notice, self.style_list["notice_content"]))
        
    def the_appendix(self):
        """附录"""
        
        appendix_style = {"fontSize":16,"textColor":"#0b9c9e","spaceAfter":20,"spaceBefore":20,"alignment":TA_CENTER,"bulletFontName":"Symbol"}
        self.add_style_sheet("appendix_style",appendix_style)
        appendix_info1 = '附录'
        self.Elements.append(Paragraph(appendix_info1, self.style_list["appendix_style"]))
        
        #检测质量控制
        appendix_title1_style = {"fontSize":14,"textColor":"#3f3f3f","spaceAfter":20,"spaceBefore":20,"alignment":TA_LEFT,"bulletFontName":"Symbol"}
        self.add_style_sheet("appendix_title1",appendix_title1_style)
        appendix_info_title1 = '检测质量控制'
        self.Elements.append(Paragraph(appendix_info_title1, self.style_list["appendix_title1"]))
        
        #1. 全流程质控
        appendix_title2_style = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":20,"spaceBefore":20,"alignment":TA_LEFT,"bulletFontName":"Symbol"}
        self.add_style_sheet("appendix_title2",appendix_title2_style)
        appendix_info_title1 = '1. 全流程质控'
        self.Elements.append(Paragraph(appendix_info_title1, self.style_list["appendix_title2"]))
        
        table_contents = [
            ["内参"  ,"合格" ],
            ["质控品" , "合格"],
            ["防错标签（UMSI）" ,"合格"]
        ]
        table_data = []
        table_data.append([Paragraph('质控参数', self.style_list["result_abstract_header"]), Paragraph('质控结果', self.style_list["result_abstract_header"])])
        for items in table_contents:
            tmp = []
            for content in items:
                tmp.append(Paragraph(content, self.style_list["result_abstract_content"]))
            table_data.append(tmp)
        
        table_style = [
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
            
            ('BACKGROUND', (0, 0), (-1, 0), '#0b9c9e'),  # 设置背景颜色
            ('GRID', (0, 0), (-1, -1), 0.05, colors.gray),  # 设置表格框线为灰色，线宽为0.1
            ('BOX',(0,0),(-1,-1),0.1,colors.gray),
        ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table) 
        
        
        #2. 测序数据质控
        appendix_info_title2 = '2. 测序数据质控'
        self.Elements.append(Paragraph(appendix_info_title2, self.style_list["appendix_title2"]))
        
        table_contents = [
             "60924229" ,"1086504400", "92.66" ,"合格"
        ]
        
        table_data = [
                    [Paragraph('总序列（reads）数', self.style_list["result_abstract_header"]), 
                     Paragraph('总碱基数', self.style_list["result_abstract_header"]),
                     Paragraph('Q30 比例（%）', self.style_list["result_abstract_header"]), 
                     Paragraph('测序质量评估', self.style_list["result_abstract_header"])],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table) 
        
        
        notice_content = [
            "注：",
            "1. 总序列（reads）数：样本经过高通量测序并过滤低质量数据后，获得的所有测序序列的数目。",
            "2. 总碱基数：样本经过高通量测序并过滤低质量数据后，获得的所有测序序列的总碱基的数目。",
            "3. 碱基质量值（Q）是衡量测序质量的重要指标，Q越高代表碱基被测错的概率越小，其中Q30为测序错误的概率在0.1%的 质量值。Q30比例为样本测序数据中质量值≥Q30的碱基比例。"
        ]
        for notice in notice_content:
            self.Elements.append(Paragraph(notice, self.style_list["notice_content"]))
        
        #检测分析性能
        appendix_info_title2 = '检测质量控制'
        self.Elements.append(Paragraph(appendix_info_title2, self.style_list["appendix_title1"]))
        
        table_contents = [
             "50-500 copies/mL" ,"50 copies/mL", "99%"
        ]
        
        table_data = [
                    [Paragraph('检测限', self.style_list["result_abstract_header"]), 
                     Paragraph('灵敏度', self.style_list["result_abstract_header"]),
                     Paragraph('特异性', self.style_list["result_abstract_header"])],
                    [Paragraph(item, self.style_list["result_abstract_content"]) for item in table_contents],
                    ]
        table = Table(data=table_data, style=table_style)
        self.Elements.append(table) 
        
        #检测方法简介
        appendix_info_title3 = '检测方法简介'
        self.Elements.append(Paragraph(appendix_info_title3, self.style_list["appendix_title1"]))
        
        contents = [
            "病原宏基因组学是一种新的不依赖于培养的广泛分析临床样本中微生物组的高通量测序方法[1]。适用于不明原因发热、疑难危重以及免疫缺陷感染患者[2-3]，目前已有多篇用于脓毒症、脑膜炎、呼吸系统感染等方面的报道[4-6]。本检测基于高通量测序平台，数据库涵盖19036种微生物（9855种细菌、6926种病毒、1582种真菌、312种寄生虫、分枝杆菌复合群中的177种常见致病菌和184种支原体/衣原体），通过提取临床样本中的核酸序列，构建测序文库，对样本中核酸序列进行测序，再通过微生物专用数据库进行比对分析[9经过智能化算法获得疑似致病微生物的种属信息，并提供全面深入的报告参数，辅助临床诊疗决策[7]。",
        ]
        content_declar1 = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":5,"spaceBefore":5,"firstLineIndent":24,"leading":24}
        self.add_style_sheet("content_declar1",content_declar1)
        for content in contents:
            self.Elements.append(Paragraph(content, self.style_list["content_declar1"]))
        
    def the_references(self):
        """主要参考文献"""
        appendix_info2 = '主要参考文献'
        self.Elements.append(Paragraph(appendix_info2, self.style_list["appendix_style"]))
        
        references_content_style = {"fontSize":10,"textColor":"#3f3f3f","spaceAfter":0,"spaceBefore":0,"alignment":TA_LEFT,"LEADING":10}
        self.add_style_sheet("references_content",references_content_style)
        
        table_contents = [
            ["1" ,"Wilson, M.R., et al., Actionable diagnosis of neuroleptospirosis by next-generation sequencing. N Engl J Med, 2014.370(25): p.2408-17."],
            ["2" , "Dekker, J.P., Metagenomics for Clinical Infectious Disease Diagnostics Steps Closer to Reality. J Clin Microbiol, 2018.56(9)."],
            ["3" ,"Thoendel, M.J., et al., Identification of Prosthetic Joint Infection Pathogens Using a Shotgun Metagenomics Approach. Clin Infect Dis, 2018.67(9): p.1333-1338."],
            ["4","Brown, J.R., T. Bharucha, and J. Breuer, Encephalitis diagnosis using metagenomics: application of next generation sequencing for undiagnosed cases. J Infect, 2018.76(3): p.225-240."],
            ["5","Parize, P., et al., Untargeted next-generation sequencing-based first-line diagnosis of infection in immunocompromised adults: a multicentre, blinded, prospective study. Clin Microbiol Infect, 2017.23(8): p.574 e1-574e6."],
            ["6","Blauwkamp, T. A., et al., Analytical and clinical validation of a microbial cell-free DNA sequencing test for infectious disease. Nat Microbiol, 2019,4(4), p.663."],
            ["7","Miller, S., et al. Laboratory validation of a clinical metagenomic sequencing assay for pathogen detection in cerebrospinal fluid. Genome Res,2019."],
            ["8","Langelier ,C., et al. Integrating host response and unbiased microbe detection for lower respiratory tract infection diagnosis in critically ill adults. P Natl Acad of Sci USA, 2018,115(52):E12353-E12362."],
            ["9","Gu, W., S. Miller, and C.Y. Chiu, Clinical Metagenomic Next-Generation Sequencing for Pathogen Detection. Annu Rev Pathol, 2019. 14: p.319-338."],
            ["10","Miao, Q., et al., Microbiological Diagnostic Performance of Metagenomic Next-generation Sequencing When Applied to Clinical Practice. Clin Infect Dis, 2018. 67(suppl_2): p.S231-S240."],
            ["11","Xie, Y., et al., Next generation sequencing for diagnosis of severe pneumonia: China,2010-2018. J Infect, 2019. 78(2): p.158-169."]
        ]
        
        table_data = []
    
        for items in table_contents:
            tmp = []
            for content in items:
                tmp.append(Paragraph(content, self.style_list["references_content"]))
            table_data.append(tmp)
        
        
        table_style = [
           
            ('FONTNAME', (0, 0), (-1, -1), 'SimSun'),  # 字体
            ('FONTSIZE', (0, 0), (-1, -1), 10),  # 字体大小
            ('LEADING',(0, 0),(-1, -1), 24),  # 
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 所有表格左右中间对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格上下居中对齐
            ('LINEABOVE',(0,1),(-1,-1),0.5,"#DAEEF3"), #设置中间边框
            ('LINEABOVE',(0,0),(1,0), 4 , "#0b9c9e"),  # 设置上边框
            ('LINEBELOW',(0,-1),(1,-1), 2 , "#0b9c9e"),  # 设置下边框
        ]
        table = Table(data=table_data, style=table_style,colWidths=(1*cm,16*cm),rowHeights=1.5*cm)
        self.Elements.append(table)
        
        
    def the_notice(self):
        """检测报告使用须知"""
        appendix_info3 = '检测报告使用须知'
        self.Elements.append(Paragraph(appendix_info3, self.style_list["appendix_style"]))
        self.Elements.append(Spacer(240, 10))  # 添加空白行
        
        notice_content = {"fontSize":12,"textColor":"#3f3f3f","spaceAfter":15,"spaceBefore":20,"firstLineIndent":0, "leading":25}
        self.add_style_sheet("notice_content",notice_content)
        contents = [
            "1. 本检测报告仅对本次送检样本负责。",
            "2. 样本采集，保存，运输等环节的不当操作可能影响本产品检测性能。",
            "3. 本检测结果仅对该次送检样本负责。方法学的技术局限性、微生物载量、当前数据库构建局限或临床用药等因素均可能使得检测结果未涵盖样本中所有的微生物。",
            "4. 本检测非临床常规检测项目，仅供临床参考，临床相关解释须咨询临床医生。任何药物或治疗方案的 制定均需由主治医生结合患者的其他情况，综合考虑。",
            "5. 如对检测结果有疑问，请于收到报告后 7 个工作日内与我们联系。",
        
        ]
        for content in contents:
            self.Elements.append(Paragraph(content, self.style_list["notice_content"]))
        
        
        
        
    def run(self):
        # 生成首页
        self.home_page()
        self.next_page()
        self.declaration_page()
        self.Elements.append(NextPageTemplate("normal"))
        self.next_page()
        self.base_info()
        self.next_page()
        self.test_result_detail()
        self.next_page()
        self.test_by_declare()
        self.next_page()
        self.the_appendix()
        self.next_page()
        self.the_references()
        self.next_page()
        self.the_notice()
        
        
        self.create_doc()
       

In [32]:
rid = int(time.time())
elp = EtiologyReport(str(rid))
elp.run()

In [484]:
# 添加一个基础信息表格
# table = self.add_base_info_table()
# self.Elements.append(table)
#  # 添加一个空行
# title = ''
# self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
#  # 添加一个空行
# title = ''
# self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
# # 添加一个标题一的图片
# image = Image("./2.jpg",hAlign="LEFT")
# image.drawWidth = 250
# image.drawHeight = 40
# self.Elements.append(image)
#  # 添加一个空行
# title = ''
# self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
#  # 添加一个空行
# title = ''
# self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
# # 添加一个处理结果的图片
# image = Image("./3.jpg",hAlign="CENTER")
# image.drawWidth = 450
# image.drawHeight = 150
# self.Elements.append(image)
# title = ''
# self.Elements.append(Paragraph(title, self.style_list["BodyText"]))
#  # 添加一个空行
# # 添加一段文字说明
# contents = '评价: 高于4.36%的人群-您的微生态多样性属于中等偏低水平,提示您的肠道菌群多样性 \n 不高。肠道菌群丰富度高,是健康肠道菌群的重要特征。'
# self.Elements.append(Paragraph(contents, self.style_list["BodyText"]))

In [690]:
import pandas as pd

In [ ]:
pd.